In [1]:
import sys
sys.path.insert(0,'/home/djl34/kl_git/scripts')
from notebook_imports import *

client = Client()

/home/djl34/mambaforge/envs/raklette/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(
/home/djl34/mambaforge/envs/raklette/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36201 instead
  warnings.warn(


# in Chromosome 22

In [27]:
chrom = 22
filename = os.path.join(KL_data_dir, f"whole_genome/p/{chrom}/_metadata")
rate = dd.read_parquet(filename.split("_metadata")[0])
rate_neutral = rate[~rate["Neutral"].isna()].compute()
rate_neutral = rate_neutral.reset_index(drop = True)

In [44]:
rate_neutral["Window_50"] = rate_neutral.Pos//50

In [45]:
rate_neutral

,Pos,mu,Allele_ref,Allele,mu_index,AN_total_interpolate,AC_total,filter_int,AF_gnomADv3,AF_ukb,AF,window_100bp,polymorphic,polymorphic_100bp,Neutral,cds,Freq_bin_9,Freq_bin_10,p_9,E[p_9],Var[p_9],p_9_reverse_log,E[p_9_reverse_log],Var[p_9_reverse_log],p_2,E[p_2],Var[p_2],Window_50
0,26886471,0.117,A,G,10,1135326.0,3.0,4,0.00002,NaN,0.000003,156333,1,44,1.0,False,3,3,-0.932277,-0.779518,0.006429,-2.298098e+00,-0.541898,1.034092,-1.0,-0.266020,0.195253,537729
1,26886471,0.013,A,T,0,1135290.0,0.0,4,NaN,NaN,0.000000,156333,0,44,1.0,False,0,0,-0.965322,-0.966130,0.000019,0.000000e+00,-0.140227,0.578140,-0.0,-0.034678,0.033475,537729
2,26886471,0.020,A,C,1,1135290.0,0.0,4,NaN,NaN,0.000000,156333,0,44,1.0,False,0,0,-0.946241,-0.948165,0.000070,-5.551115e-16,-0.194174,0.710929,-0.0,-0.053759,0.050869,537729
3,26886472,0.051,A,G,4,1135290.0,0.0,4,NaN,NaN,0.000000,156333,0,44,1.0,False,0,0,-0.872834,-0.883462,0.000841,-2.220446e-16,-0.351029,0.955836,-0.0,-0.127166,0.110995,537729
4,26886472,0.030,A,C,2,1135290.0,0.0,4,NaN,NaN,0.000000,156333,0,44,1.0,False,0,0,-0.922983,-0.926907,0.000198,-4.440892e-16,-0.250862,0.821386,-0.0,-0.077017,0.071085,537729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2584117,49413567,0.051,C,G,4,1135282.0,0.0,4,NaN,NaN,0.000000,381604,0,45,1.0,False,0,0,-0.872834,-0.883462,0.000841,-2.220446e-16,-0.351029,0.955836,-0.0,-0.127166,0.110995,988271
2584118,49413567,0.020,C,A,1,1135282.0,0.0,4,NaN,NaN,0.000000,381604,0,45,1.0,False,0,0,-0.946241,-0.948165,0.000070,-5.551115e-16,-0.194174,0.710929,-0.0,-0.053759,0.050869,988271
2584119,49413568,0.030,A,C,2,1135282.0,0.0,4,NaN,NaN,0.000000,381604,0,45,1.0,False,0,0,-0.922983,-0.926907,0.000198,-4.440892e-16,-0.250862,0.821386,-0.0,-0.077017,0.071085,988271
2584120,49413568,0.020,A,T,1,1135282.0,0.0,4,NaN,NaN,0.000000,381604,0,45,1.0,False,0,0,-0.946241,-0.948165,0.000070,-5.551115e-16,-0.194174,0.710929,-0.0,-0.053759,0.050869,988271


In [46]:
rate_neutral_groups = rate_neutral.groupby("Window_50")

bin_list = [2, 9, "9_reverse_log"]

columns_header_list = ["p_{bin_number}", "E[p_{bin_number}]", "Var[p_{bin_number}]"]

header_list = [x.format(bin_number = bin_number) for x in columns_header_list for bin_number in bin_list]

rate_neutral_groups_sum = rate_neutral_groups[header_list].sum()


rate_neutral_groups_sum["Pos_start"] = rate_neutral_groups["Pos"].min()
rate_neutral_groups_sum["Pos_end"] = rate_neutral_groups["Pos"].max()

rate_neutral_groups_sum["n_sites"] = rate_neutral_groups.size()

# remove last column, since it has less than 50 sites
# rate_neutral_groups_sum = pd.DataFrame(rate_neutral_groups_sum).iloc[:-1]

for freq_bin in bin_list:
    rate_neutral_groups_sum[f"Zscore_{freq_bin}"] = ((rate_neutral_groups_sum[f"p_{freq_bin}"] - rate_neutral_groups_sum[f"E[p_{freq_bin}]"])/
                                                np.sqrt(rate_neutral_groups_sum[f"Var[p_{freq_bin}]"]))

freq_bin = 2
rate_neutral_groups_sum["Zscore_Gnocchi"] = ((rate_neutral_groups_sum[f"p_{freq_bin}"] - rate_neutral_groups_sum[f"E[p_{freq_bin}]"])/
                                                np.sqrt(np.absolute(rate_neutral_groups_sum[f"E[p_{freq_bin}]"])))

rate_neutral_groups_sum_zscore = pd.melt(rate_neutral_groups_sum, value_vars=['Zscore_2', 'Zscore_9', 'Zscore_9_reverse_log', "Zscore_Gnocchi"], value_name = "Zscore", var_name = "type")

In [64]:
rate_neutral_groups_sum = rate_neutral_groups_sum[rate_neutral_groups_sum["n_sites"] > 50]

In [71]:
method_list = ["Zscore_2", "Zscore_9", "Zscore_9_reverse_log", "Zscore_Gnocchi"]

list_of_list = []

autocorr_list = [1, 2, 3, 10, 30, 100, 200]

for method in method_list:

    row = []
    
    row.append(method)

    for autocorr in autocorr_list:
        row.append(rate_neutral_groups_sum[method].autocorr(autocorr))

    list_of_list.append(row)

In [72]:
df = pd.DataFrame(list_of_list, columns = ["Method"] + [f"Autocorr_{autocorr}" for autocorr in autocorr_list ])

df

,Method,Autocorr_1,Autocorr_2,Autocorr_3,Autocorr_10,Autocorr_30,Autocorr_100,Autocorr_200
0,Zscore_2,0.149729,0.141971,0.123385,0.116629,0.080498,0.035753,0.006872
1,Zscore_9,0.161990,0.149177,0.130321,0.116679,0.061301,0.019692,-0.003481
2,Zscore_9_reverse_log,0.165378,0.156811,0.130641,0.120384,0.082579,0.049188,0.009644
3,Zscore_Gnocchi,0.144202,0.137532,0.118479,0.112074,0.076292,0.033634,0.007284


In [59]:
list_of_list

[['Zscore_2',
  0.14676750059824983,
  0.14037169899596147,
  0.1221097290703449,
  0.11308306307954341],
 ['Zscore_9',
  0.16073362106553085,
  0.14640209503840163,
  0.12756799846015188,
  0.11340666441863011],
 ['Zscore_9_reverse_log',
  0.16300579699474058,
  0.15614889661867054,
  0.1315522172794584,
  0.11774427334734877],
 ['Zscore_Gnocchi',
  0.14102151913888344,
  0.1360874041251483,
  0.11720632767386167,
  0.10861411120831688]]

In [15]:
method_list = ["Zscore_2", "Zscore_9", "Zscore_9_reverse_log", "Zscore_Gnocchi", "Zoonomia_phastcons_primates_mean", "phyloP_mean"]

for method in method_list:
    print(method)
    print("\t", rate[method].autocorr())
    print("\t", rate[method].autocorr(200))

Zscore_2
	 0.16988071416315892
	 0.010668545730015036
Zscore_9
	 0.15745146559874562
	 0.012088494392101889
Zscore_9_reverse_log
	 0.17889085092216903
	 0.014425048601347471
Zscore_Gnocchi
	 0.16494210124234546
	 0.011086569780729937
Zoonomia_phastcons_primates_mean
	 0.6178393352463266
	 0.07962488739543222
phyloP_mean
	 0.7502136581847505
	 0.09063410365583757


## Fraction of cds

In [16]:
chrom_set = all_chrom_set

filename = os.path.join(KL_data_dir, "whole_genome/windows/50bp/{chrom}.tsv")
input_list = [filename.format(chrom = chrom) for chrom in chrom_set]

rate_df = []

for filename in input_list:
    rate = pd.read_csv(filename, sep = "\t")
    rate_df.append(rate)

In [20]:
method_list = ["Zscore_2", "Zscore_9", "Zscore_9_reverse_log", "Zscore_Gnocchi", "Zoonomia_phastcons_primates_mean", "phyloP_mean"]

i = 1

chrom_list = []
autocorr_list = []
autocorr200_list = []

for rate in rate_df:
    for method in method_list:
        chrom_list.append(i)
        autocorr_list.append(rate[method].autocorr())
        autocorr200_list.append(rate[method].autocorr(200))

    i += 1

In [24]:
df = pd.DataFrame(zip(chrom_list, autocorr_list, autocorr200_list, method_list * len(rate_df)), columns = ["Chrom", "Autocorr", "Autocorr_200", "method"])

In [26]:
df.pivot(index='Chrom', columns='method', values='Autocorr')

method,Zoonomia_phastcons_primates_mean,Zscore_2,Zscore_9,Zscore_9_reverse_log,Zscore_Gnocchi,phyloP_mean
Chrom,,,,,,
1,0.621000,0.106892,0.099046,0.118448,0.104407,0.785334
2,0.638994,0.113281,0.094664,0.119189,0.110741,0.814635
3,0.615048,0.114419,0.095783,0.119373,0.111793,0.791032
4,0.605882,0.112678,0.094394,0.118937,0.109784,0.783621
5,0.631235,0.111224,0.096232,0.115750,0.108749,0.808760
6,0.618032,0.161417,0.136341,0.180480,0.156665,0.785799
7,0.621162,0.117326,0.102686,0.125066,0.114071,0.792137
8,0.623227,0.144671,0.117894,0.150981,0.138136,0.796353
9,0.625529,0.154605,0.122184,0.161425,0.151152,0.802264
